### Import Libraries and Packages

In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [3]:
df = pd.read_csv('/content/covid_toy.csv')
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [4]:
# check for nulls
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [5]:
from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test=train_test_split(df.iloc[:,0:5],df.iloc[:,-1],test_size=0.2,random_state=40)
X_train, X_test, y_train, y_test=train_test_split(df.drop(columns=['has_covid']), df['has_covid'], test_size=0.2, random_state=40)

In [6]:
X_train.head()

,age,gender,fever,cough,city
72,83,Female,101.0,Mild,Kolkata
66,51,Male,104.0,Mild,Kolkata
69,73,Female,103.0,Mild,Delhi
67,65,Male,99.0,Mild,Bangalore
26,19,Female,100.0,Mild,Kolkata


### Imputation and Transformation in Normal Way

#### Impute data for 'fever' feature

In [7]:
# adding simple imputer to fever col
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])

# also the test data
X_test_fever = si.fit_transform(X_test[['fever']])
                                 
X_train_fever.shape

(80, 1)

#### Perform Ordinal Encoding for 'cough' feature

In [8]:
# Ordinalencoding -> cough
oe = OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])

# also the test data
X_test_cough = oe.fit_transform(X_test[['cough']])

X_train_cough.shape

(80, 1)

#### Perform Ordinal Encoding for 'gender' and 'city' features

In [9]:
# OneHotEncoding -> gender,city
ohe = OneHotEncoder(drop='first',sparse=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])

# also the test data
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])

X_train_gender_city.shape

(80, 4)

#### Extract 'age' feature

In [10]:
# Extracting Age
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

X_train_age.shape

(80, 1)

#### Concatenate the transformed data numpy arrays and 'age' data

In [11]:
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)

# also the test data
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)

X_train_transformed.shape

(80, 7)

### Using Scikit Learn's Column Transformer

In [12]:
from sklearn.compose import ColumnTransformer

In [13]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(sparse=False,drop='first'),['gender','city'])
],remainder='passthrough')

In [14]:
transformer.fit_transform(X_train).shape

(80, 7)

In [15]:
transformer.transform(X_test).shape

(20, 7)